<a href="https://colab.research.google.com/github/mungaimoureen/Week-4-IP-project/blob/main/Moringa_Autolib_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Cleaning and Analysis

In [ ]:
#Loading our Data from pandas, numpy, Matlib, seaborn.

In [17]:
#Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#Set plotting parameters
sns.set()
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Import files
from google.colab import files

In [ ]:
# Load dataset
autolib_df = pd.read_csv('/content/Autolib_dataset.csv', encoding='utf8')

In [ ]:
# Previewing data
autolib_df.head()

In [ ]:
autolib_df.tail()

In [ ]:
autolib_df.sample(10)

In [ ]:
# Investigate data
autolib_df.info()

In [ ]:
# Shape of dataset
autolib_df.shape

The comments and scheduel at columns are not filled

In [ ]:
# Drop nearly empty columns
autolib_df.drop(['Displayed comment', 'Scheduled at'], axis=1, inplace=True)

In [ ]:
# Rename columns
col_names = autolib_df.columns.tolist()
for col in range(len(col_names)):
  col_names[col] = col_names[col].lower().replace(" ", "_")

autolib_df.columns = col_names

In [ ]:
# Check for duplicates
autolib_df.duplicated().any()

There were no duplicates from the data.

In [ ]:
# Check for missing values
autolib_df.isnull().sum().sort_values(ascending=False)

No null values detected in the data


In [ ]:
# Check column datatypes
autolib_df.dtypes

Exploring our Data.

In [ ]:
# Which cities are available in our dataset?
autolib_df.city.unique()

In [ ]:
# Extract paris data
paris_df = autolib_df[autolib_df.city == 'Paris']

In [ ]:
# The paris dataframe preview
paris_df.shape

In [ ]:
paris_df.head()

In [ ]:
# Create time
paris_df['time'] = paris_df[paris_df.columns[21:23]].apply(lambda x: ":".join(x.astype(str)), axis = 1)

In [ ]:
# Create date 
paris_df['date'] = paris_df[paris_df.columns[18:21]].apply(lambda x: '/'.join(x.astype(str)), axis=1)

In [ ]:
#Combine datetime 
paris_df['datetime'] = paris_df[paris_df.columns[23:]].apply(lambda x: " ".join(x.astype(str)), axis = 1)

In [ ]:
# Coverting datetime
paris_df.datetime = pd.to_datetime(paris_df.datetime)

QUERING.

In [ ]:
# How many unique stations are there in paris?
paris_df.id.unique()

In [ ]:
# Investigate kind variable 
paris_df.kind.value_counts().plot(kind='bar')

Output explanation.
Station and space has all the resources.
parking has no charging slots,
centre do not have no resources.


In [ ]:
# rental status
paris_df.rental_status.value_counts()

Resources are only available when "operational", except for "broken" which will have Slots, but other resources that is, Bluecars, utilib or charging slots.


In [ ]:
# station type
paris_df['station_type'].value_counts()

Resources available at sub_centers.

In [ ]:
# station status
paris_df['status'].value_counts()

There are no resources available for "scheduled", which is the status if there is a "scheduled at" date but there can be resources associated with "closed" stations.

In [ ]:
# charging status
paris_df.charging_status.value_counts()

A functioning station has the following resources:
Blue cars
Utilib / Utilib 1.4
Charging slots
Parking slots



In [ ]:
#ruling out stations with center kind
available_stations = available_stations[available_stations.kind != 'CENTER']

In [ ]:
# ruling out stations with a scheduled status
available_stations = available_stations[available_stations.status != 'scheduled']

In [ ]:
# ruling out stations with subs_center station type
available_stations = available_stations[available_stations.station_type != 'subs_center']

In [ ]:
# ruling out stations without operational rental status
available_stations = paris_df[paris_df.rental_status == 'operational']

In [ ]:
# available_stations.shape

In [ ]:
# how many times does each station occurs in the dataset
station_occurences = available_stations.id.value_counts(ascending=False)
station_occurences

Eliminate stations that only appear once. have not much relevance.

In [ ]:
#  stations that occur more than once
stations_active = station_occurences[station_occurences > 1 ].index.tolist()

In [ ]:
# stations that are onli active
available_stations = available_stations[available_stations['id'].isin(stations_active)]

In [ ]:
available_stations.head()

In [ ]:
# A dataframe with blue car counter data
blue_car_flow = available_stations[['postal_code','bluecar_counter','id','datetime']]

In [ ]:
# Sorting data by id and datetime column
blue_car_flow = blue_car_flow.sort_values(by=['id','datetime'])

In [ ]:
blue_car_flow.head()

In [ ]:
# Creating a column which constitutes difference in subsequent rows
blue_car_flow['diff'] = blue_car_flow.groupby(['id'])['bluecar_counter'].diff().fillna(0)

In [ ]:
blue_car_flow

A positive value indicates a **dropoff** as the number of blue cars available increases, while a negative value indicates a **pickup** as the number of blue cars available decreases.

In [ ]:
# We need an hour variable
blue_car_flow['hour'] = blue_car_flow.datetime.dt.hour

In [ ]:
# Creating dataframe consisting of only pickups
bluecar_pickups = blue_car_flow[blue_car_flow['diff'] < 0]

In [ ]:
bluecar_pickups.head()

In [ ]:
bluecar_pickups.shape

In [ ]:
# To get the most popular hour we will look at the frequency of each hour in the pickups data
bluecar_pickups.hour.value_counts().sort_values(ascending=False)

In [ ]:
sns.set(style='whitegrid')
y_ax =  bluecar_pickups.hour.value_counts().sort_values(ascending=False)
x_ax =  y_ax.index
frequency = plt.figure()
sns.barplot(x= x_ax, y= y_ax, palette='pastel')
plt.xlabel('Hour of the day')
plt.ylabel('Pickup Frequency')
plt.title('Pickup frequency per hour of the day',fontdict={'fontsize':15})
plt.savefig("frequency.png")
files.download('frequency.png')

From the graph above, most pickups are at 9:00 pm - 10:00 pm

In [ ]:
# Creating dataframe consisting of only drop offs
bluecar_dropoffs = blue_car_flow[blue_car_flow['diff'] > 0]

In [ ]:
bluecar_dropoffs.head()

In [ ]:
# Most popular hour for dropoffs
bluecar_dropoffs.hour.value_counts().sort_values(ascending=False)

In [ ]:
# Plotting dropoff frequency
y_val = bluecar_dropoffs.hour.value_counts().sort_values(ascending=False)
x_val = y_val.index
sns.barplot(x = x_val, y=y_val, palette='pastel')
plt.xlabel('Hour of the day')
plt.ylabel('Dropoff frequency')
plt.title('Dropoff frequency per hour of the day',fontdict={'fontsize':15})
plt.savefig("dropoff.png")
files.download('dropoff.png')

Most drop offs are between 12:00 - 1:00 pm